# Activity 8: Re-training a model dynamically
In this activity, we re-train our model every time new data is available.

First, we start by importing `cryptonic`. Cryptonic is a simple software application developed for this course that implements all the steps up to this section using Python classes and modules. Consider Cryptonic a template on how you could develop similar applications.

In [1]:
from cryptonic import Model
from cryptonic import CoinMarketCap
from cryptonic.models import normalizations

from tqdm import tqdm_notebook

%autosave 5
%matplotlib inline

Using TensorFlow backend.
C:\Users\DELL\anaconda3\envs\deploy_env\lib\site-packages\tensorflow\python\framework\dtypes.py:469: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\DELL\anaconda3\envs\deploy_env\lib\site-packages\tensorflow\python\framework\dtypes.py:470: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\DELL\anaconda3\envs\deploy_env\lib\site-packages\tensorflow\python\framework\dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\DELL\anaconda3\envs\deploy

Autosaving every 5 seconds


## Fetching Real-time Data
Throughout this project we have been using data originally provided by [CoinMarketCat](https://coinmarketcap.com/). We have created an interface for collecting both real-time and historical data as as part of `cryptonic`: the class `CoinMarketCap()`.

In [2]:
print(CoinMarketCap())



        Crypto-currency data comes from the website CoinMarketCap.
        CoinMarketCap is can be accessed at: https://coinmarketcap.com/

        The permission to use the data is available on their FAQ

            https://coinmarketcap.com/faq/

        and reads:

            "Q: Am I allowed to use content (screenshots, data, graphs, etc.) 
            for one of my personal projects and/or commercial use?

            R: Absolutely! Feel free to use any content as you see fit. 
            We kindly ask that you cite us as a source."
        
        


Our model is designed to work with daily data. Let's go ahead and collect historic daily data from CoinMarketCap (this is the same data used previously).

In [3]:
historic_data = CoinMarketCap.historic()
historic_data.head()

date      open      high       low     close       volume  \
0  2020-10-29  13271.30  13612.05  12980.06  13437.88  56499499598   
1  2020-10-28  13654.21  13837.70  12932.25  13271.29  35867318895   
2  2020-10-27  13075.24  13759.67  13060.84  13654.22  33749878156   
3  2020-10-26  13031.20  13225.30  12822.38  13075.25  29461458313   
4  2020-10-25  13108.06  13329.18  12910.06  13031.17  24406920575   

     market_cap iso_week  
0  248995322466  2020-44  
1  245899824465  2020-44  
2  252985946390  2020-44  
3  242250997936  2020-44  
4  241425217440  2020-43

The data contains practically the same variables from our earlier dataset. However, much of the data comes from an earlier period. Recent Bitcoin prices have gained a lot of volatility if compared to the prices of a few years ago. Before using this data in our model, let's make sure to filter it to dates after January 1, 2017.

In [4]:
#
#  Using the Pandas API, filter the dataframe
#  for observations from 2017 only. 
# 
#  Hint: use the `date` variable.
#
import datetime
model_data = historic_data.loc[historic_data['date'] > '2017-1-1']
model_data.tail()


date     open     high      low    close      volume   market_cap  \
1120  2017-10-05  4229.88  4362.64  4164.05  4328.41  1161769984  71876277937   
1121  2017-10-04  4319.37  4352.31  4210.42  4229.36  1116770048  70225033578   
1122  2017-10-03  4408.46  4432.47  4258.89  4317.48  1288019968  71681069637   
1123  2017-10-02  4395.81  4470.23  4377.46  4409.32  1431730048  73195646776   
1124  2017-10-01  4341.05  4403.74  4269.81  4403.74  1208210048  73094870674   

     iso_week  
1120  2017-40  
1121  2017-40  
1122  2017-40  
1123  2017-40  
1124  2017-39

In [5]:
# model_data.date = model_data.date.astype(str)

In [6]:
model_data.head()

date      open      high       low     close       volume  \
0  2020-10-29  13271.30  13612.05  12980.06  13437.88  56499499598   
1  2020-10-28  13654.21  13837.70  12932.25  13271.29  35867318895   
2  2020-10-27  13075.24  13759.67  13060.84  13654.22  33749878156   
3  2020-10-26  13031.20  13225.30  12822.38  13075.25  29461458313   
4  2020-10-25  13108.06  13329.18  12910.06  13031.17  24406920575   

     market_cap iso_week  
0  248995322466  2020-44  
1  245899824465  2020-44  
2  252985946390  2020-44  
3  242250997936  2020-44  
4  241425217440  2020-43

## The `Model()` Class
We have also created the class `Model()` which compiles all the code we have written so far. We will use that class to build, train, and evaluate our model.

In [7]:
model_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1125 entries, 0 to 1124
Data columns (total 8 columns):
date          1125 non-null object
open          1125 non-null float64
high          1125 non-null float64
low           1125 non-null float64
close         1125 non-null float64
volume        1125 non-null int64
market_cap    1125 non-null int64
iso_week      1125 non-null object
dtypes: float64(4), int64(2), object(2)
memory usage: 79.1+ KB


In [8]:
M = Model(data=model_data,
          variable='close',
          predicted_period_size=7)

In [9]:
M.build()

In [10]:
M.train(epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 1s 672ms/step - loss: 0.0233
Epoch 2/100
1/1 [==============================] - 0s 59ms/step - loss: 0.0216
Epoch 3/100
1/1 [==============================] - 0s 50ms/step - loss: 0.0204
Epoch 4/100
1/1 [==============================] - 0s 50ms/step - loss: 0.0196
Epoch 5/100
1/1 [==============================] - 0s 60ms/step - loss: 0.0188
Epoch 6/100
1/1 [==============================] - 0s 58ms/step - loss: 0.0181
Epoch 7/100
1/1 [==============================] - 0s 49ms/step - loss: 0.0175
Epoch 8/100
1/1 [==============================] - 0s 47ms/step - loss: 0.0170
Epoch 9/100
1/1 [==============================] - 0s 52ms/step - loss: 0.0165
Epoch 10/100
1/1 [==============================] - 0s 52ms/step - loss: 0.0160
Epoch 11/100
1/1 [==============================] - 0s 49ms/step - loss: 0.0155
Epoch 12/100
1/1 [==============================] - 0s 50ms/step - loss: 0.0150
Epoch 13/100
1/1 [==============================

We can now use the model for making predictions with the `predict()` method. The parameter `denormalized` will return values in the original scale of the data. In our case, US dollars.

In [11]:
M.predict(denormalized=True)

array([13468.672, 11937.052, 11799.343, 11794.85 , 11386.053, 10962.978,
       10794.767], dtype=float32)

We now evaluate our model to inspect the statistics for the last epoch of training compared to a single test week.

In [12]:
M.evaluate()

{'mse': 0.0, 'rmse': 1934.45, 'mape': 13.03}

Finally, we can now save the trained model on disk for later use.

In [13]:
M.save('bitcoin_model_prod_v0.h5')

Our `Model()` class can also load a previously trained model when instantiated with the `path` parameter.

In [14]:
M = Model(path='bitcoin_model_prod_v0.h5',
          data=model_data,
          variable='close',
          predicted_period_size=7)

In [15]:
M.predict(denormalized=True)

array([13468.672, 11937.052, 11799.343, 11794.85 , 11386.053, 10962.978,
       10794.767], dtype=float32)

## New Data, Re-train Old Model
One strategy discussed earlier regards the re-training of our model with new data. In our case, our biggest concern is to shape data in a way that the model has been configured. As an example, we will configure our model to predict a week using 40 weeks. We will first train the model with the first 40 weeks of 2017, then continue to re-train it over the following weeks until we reach week 50.

First, let's build a model with the first set of data. Notice how we use `7*40 + 7` as the indexer. This is because we use 40 weeks for training and 1 week for testing. 

In [16]:
M1 = Model(data=model_data[0*7:7*44 + 7],
          variable='close',
          predicted_period_size=7)

In [17]:
M1.build()

In [18]:
M1.train()

In [19]:
#
#  Complete the range function and
#  the model_data filtering parameters
#  using an index to split the data in overlapping
#  groups of 7 days. Then, re-train our model
#  and collect the results.
#
#  The variables A, B, C, and D are placeholders.
#
results = []
for i in range(0, 10):
    M1.train(model_data[i* 7:7*(44+i) + 7])
    results.append(M.evaluate())

In [20]:
for i, result in enumerate(results):
    print(f'Week {40+i+1}: {result}')

Week 41: {'mse': 0.0, 'rmse': 1934.45, 'mape': 13.03}
Week 42: {'mse': 0.0, 'rmse': 1934.45, 'mape': 13.03}
Week 43: {'mse': 0.0, 'rmse': 1934.45, 'mape': 13.03}
Week 44: {'mse': 0.0, 'rmse': 1934.45, 'mape': 13.03}
Week 45: {'mse': 0.0, 'rmse': 1934.45, 'mape': 13.03}
Week 46: {'mse': 0.0, 'rmse': 1934.45, 'mape': 13.03}
Week 47: {'mse': 0.0, 'rmse': 1934.45, 'mape': 13.03}
Week 48: {'mse': 0.0, 'rmse': 1934.45, 'mape': 13.03}
Week 49: {'mse': 0.0, 'rmse': 1934.45, 'mape': 13.03}
Week 50: {'mse': 0.0, 'rmse': 1934.45, 'mape': 13.03}


In [21]:
M1.predict(denormalized=True)

array([11338.19  , 11365.8545, 12210.12  , 12470.185 , 12450.33  ,
       12079.48  , 12047.39  ], dtype=float32)

## New Data, New Model
Another strategy is to create and train a new model evey time new data is available. This approach tends to reduce catastrophic forgetting, but training time increases as data increases. 

It's implementation is quite simple.

Let's assume we have old data for 49 weeks of 2017 and after a week we now have new data. We represent this wtih the variables `old_data` and `new_data`. 

In [22]:
old_data = model_data[0*7:7*48 + 7]

In [23]:
new_data = model_data[0*7:7*49 + 7]

In [24]:
M2 = Model(data=old_data,
          variable='close',
          predicted_period_size=7)

In [25]:
M2.build()
M2.train()

In [26]:
M2.predict(denormalized=True)

array([13465.045, 13540.92 , 12976.685, 12871.178, 12676.115, 13332.57 ,
       13155.86 ], dtype=float32)

Now, assume that new data is available. Using this technique we go ahead and create a new model using only the new data. 

In [27]:
#
#  Re-instantiate the model with the Model()
#  class using the new_data variable instead
#  of the old_data one. 
#
M3 = Model(data=new_data,
          variable='close',
          predicted_period_size=7)


In [28]:
M3.build()
M3.train()

In [29]:
M3.predict(denormalized=True)

array([13409.588, 14106.087, 14444.852, 14626.415, 15070.057, 15010.605,
       14957.913], dtype=float32)

This approach is very simple to implement and tends to work well. We will be using this to deploy our application.